### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [4]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
end_date   = "2024-05-01"

# Build URL for CME
cme_url = f"{base_url}{CME}?startDate={startDate}&endDate={end_date}&api_key={NASA_API_KEY}"


In [ ]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(cme_url)


In [ ]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()


In [ ]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0], indent=4))



In [ ]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)

# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]

cme_df.head()

In [ ]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df.dropna(subset='linkedEvents', inplace=True)

cme_df.head()

In [ ]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []
        
# Iterate over each index in the DataFrame
for i in cme_df.index:
    activityID = cme_df.loc[i, 'activityID']
    startTime = cme_df.loc[i, 'startTime']
    linkedEvents = cme_df.loc[i, 'linkedEvents']

    # Iterate over each dictionary in the list
    for c in linkedEvents:
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
        expanded_rows.append({'activityID': activityID, 'startTime': startTime, 'linkedEvents': c})
# Create a new DataFrame from the expanded rows
expanded_cme_df = pd.DataFrame(expanded_rows)
expanded_cme_df

In [ ]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(input_dict):
    try:
        activityID = input_dict.get('activityID', None)
        return activityID
    except (ValueError, TypeError) as e:
        # Log the error or print it for debugging
        print(f"Error processing input dictionary: {input_dict}. Error: {e}")
        return None

extract_activityID_from_dict(expanded_cme_df.loc[0, 'linkedEvents'])


In [ ]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
expanded_cme_df.loc[:, 'GST_ActivityID'] = expanded_cme_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))


In [ ]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
expanded_cme_df.dropna(subset='GST_ActivityID', inplace=True)

In [ ]:
# print out the datatype of each column in this DataFrame:
print(expanded_cme_df.dtypes)


In [ ]:
# Convert the 'GST_ActivityID' column to string format 
expanded_cme_df['GST_ActivityID'] = pd.Series(expanded_cme_df['GST_ActivityID'], dtype="string")
expanded_cme_df

In [ ]:
# Get rid of any spaces in startTime
expanded_cme_df['startTime']=expanded_cme_df['startTime'].str.strip()

In [ ]:
# Convert startTime to datetime format  
expanded_cme_df['startTime'] = pd.to_datetime(expanded_cme_df['startTime'])
expanded_cme_df['startTime']

In [ ]:
# Rename startTime to startTime_CME and activityID to cmeID
expanded_cme_df.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'}, inplace=True)
# Drop linkedEvents
expanded_cme_df.drop(columns=['linkedEvents'], inplace=True)
# Verify that all steps were executed correctly
# Check the structure and data types
expanded_cme_df.info()
# Display the first few rows of the modified DataFrame
expanded_cme_df.head()

In [ ]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
expanded_cme_df = expanded_cme_df[expanded_cme_df['GST_ActivityID'].str.contains('GST')]
expanded_cme_df

### GST Data

In [ ]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
end_date   = "2024-05-01"

# Build URL for GST
gst_url = f"{base_url}{GST}?startDate={startDate}&endDate={end_date}&api_key={NASA_API_KEY}"


In [ ]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_url)


In [ ]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[0], indent=4))

In [ ]:
# Convert gst_json to a Pandas DataFrame 
gst_df = pd.DataFrame(gst_json)

# Keep only the columns: activityID, startTime, linkedEvents
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]
gst_df

In [ ]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df.dropna(subset='linkedEvents', inplace=True)

In [ ]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df = gst_df.explode('linkedEvents', ignore_index=True).dropna()
gst_df

In [ ]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_df.loc[:, 'CME_ActivityID'] = gst_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df.dropna(subset='CME_ActivityID', inplace=True)

In [ ]:
# Convert the 'CME_ActivityID' column to string format 
gst_df['CME_ActivityID'] = gst_df['CME_ActivityID'].astype(str)

# Convert the 'gstID' column to string format 
gst_df['gstID'] = gst_df['gstID'].astype(str)


In [ ]:
# Get rid of any spaces in startTime
gst_df['startTime']=gst_df['startTime'].str.strip()

In [ ]:
# Convert startTime to datetime format  
gst_df['startTime'] = pd.to_datetime(gst_df['startTime'])

In [ ]:

# Rename startTime to startTime_GST 
gst_df.rename(columns={'startTime': 'startTime_GST'}, inplace=True)

# Drop linkedEvents
gst_df.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
gst_df

In [ ]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
gst_df = gst_df[gst_df['CME_ActivityID'].str.contains('CME')]
gst_df

### Merge both datatsets

In [ ]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(gst_df, expanded_cme_df, left_on=['gstID', 'CME_ActivityID'], right_on=['GST_ActivityID', 'cmeID'])
merged_df

In [ ]:
# Verify that the new DataFrame has the same number of rows as cme and gst
merged_df.info()

### Computing the time it takes for a CME to cause a GST

In [ ]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df['timeDiff'] = (merged_df['startTime_GST'] - merged_df['startTime_CME']).dt.total_seconds()
merged_df.head()


In [ ]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
time_metrics = merged_df['timeDiff'].describe()
print(time_metrics)

### Exporting data in csv format

In [ ]:
# Export data to CSV without the index
merged_df.to_csv('cmes_gsts.csv', index=False)
